<a href="https://colab.research.google.com/github/SimonRullePaderborn/Robot/blob/main/LaufzeitenPredizierung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
liste_moeglicher_ziele=[]
for j1_winkel in range(-180,185,10):
  for j2_winkel in range(-180,185,10):
    if abs(j1_winkel)+abs(j2_winkel)<=180:
      liste_moeglicher_ziele.append([j1_winkel,j2_winkel])

In [ ]:
startposition=[0,0]
speeds=[10,30,50]

In [ ]:
Versuche=[]
for speed in speeds:
  for ziel in liste_moeglicher_ziele:
    Versuche.append(ziel+[speed])

In [ ]:
Versuche[::10]


# Einlesen der Daten

In [ ]:
!wget https://raw.githubusercontent.com/SimonRullePaderborn/Robot/main/Recorded_Data.csv

Download der Daten aus dem Github Repository. 

In [ ]:
!pip install tensorflow
!pip install keras

In [ ]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('Recorded_Data.csv', names=["j1_Degree", "j2_Degree", "Speed", "Time"])

Importieren der Daten, sowie die Zuweisung der Spaltennahmen

In [ ]:
df.head()

,j1_Degree,j2_Degree,Speed,Time
0,-90,0,10,4.826906
1,-80,-10,10,4.379949
2,-80,0,10,4.384195
3,-80,10,10,4.268559
4,-70,-20,10,3.819944


In [ ]:
print(df)

     j1_Degree  j2_Degree  Speed      Time
0          -90          0     10  4.826906
1          -80        -10     10  4.379949
2          -80          0     10  4.384195
3          -80         10     10  4.268559
4          -70        -20     10  3.819944
..         ...        ...    ...       ...
540         80          0     50  1.243056
541         80         10     50  1.244489
542         90          0     50  1.356405
543         32        -80     22  2.140223
544        -10        -34     70  0.683716

[545 rows x 4 columns]


In [ ]:
print(len(df.index))

545


In [ ]:
data = df.iloc[:-2] 

Mit "df.iloc[:-2]" werden die letzten beien Einträge aus dem Trainingsdatenset genommen. Die dort gespeicherten Werte [32,-80,22,2.140223264694214] sowie
[-10,-34,70,0.6837155818939209] habe ich manuell mit bewussten Abweichungen vom sonst regulären Messschema aufzeichnen lassen, mit dem Ziel am Ende das Netz auch auf diese, noch stärker unbekannte Form, testen zu können. Das "Minus-Zeichen" vor der 2 drückt dabei aus, dass es **letzten** zwei Zeilen sind.


In [ ]:
print(data)

     j1_Degree  j2_Degree  Speed      Time
0          -90          0     10  4.826906
1          -80        -10     10  4.379949
2          -80          0     10  4.384195
3          -80         10     10  4.268559
4          -70        -20     10  3.819944
..         ...        ...    ...       ...
538         70         20     50  1.130942
539         80        -10     50  1.244369
540         80          0     50  1.243056
541         80         10     50  1.244489
542         90          0     50  1.356405

[543 rows x 4 columns]


# Interpolieren

In [ ]:
interpolate_train = data[data.index % 11 !=0]
print('Anzahl der Daten im Trainingsset: '+ str(len(interpolate_train.index)))
print(interpolate_train)


interpolate_evaluate = data[data.index %11 == 0]
print('anzahl der Daten im Evaluateset: '+ str(len(interpolate_evaluate.index)))
print(interpolate_evaluate)

Für das Trainings-Datensatz wird jede 11. Zeile aus den Raw-Daten ausgeschlossen. Ausschließlich jede 11. wird dagegen in das Evaluierungs-Datensatz aufgenommen.

In [ ]:
array_inter_train_x = interpolate_train[['j1_Degree', 'j2_Degree', 'Speed']].to_numpy()
array_inter_train_y = interpolate_train[['Time']].to_numpy()

array_inter_evaluate_x = interpolate_evaluate[['j1_Degree', 'j2_Degree', 'Speed']].to_numpy()
array_inter_evaluate_y = interpolate_evaluate[['Time']].to_numpy()

Konvertrietung der Dataframes to Numpy-Arrays.

In [ ]:
array_inter_train_x.shape
#array_inter_train_y.shape
#array_inter_evaluate_x.shape 
#array_inter_evaluate_y.shape

(493, 3)

In [ ]:
print(array_inter_train_x)

[[-80 -10  10]
 [-80   0  10]
 [-80  10  10]
 ...
 [ 80   0  50]
 [ 80  10  50]
 [ 90   0  50]]


In [ ]:
array_inter_train_x.shape

(493, 3)

In [ ]:
print(array_inter_train_x)

[[-80 -10  10]
 [-80   0  10]
 [-80  10  10]
 ...
 [ 80   0  50]
 [ 80  10  50]
 [ 90   0  50]]


In [ ]:
print(array_inter_train_y)

In [ ]:
model_inter=keras.models.Sequential()

In [ ]:
i = 0
while i < 10:
  model_inter.add(keras.layers.Dense(4,activation='relu'))
  i = i + 1 

In [ ]:
model_inter.add(keras.layers.Dense(1,activation='linear'))

In [ ]:
model_inter.compile(loss='mse',optimizer=tf.keras.optimizers.Adam())

In [ ]:
model_inter.fit(array_inter_train_x,array_inter_train_y,epochs=1000,batch_size=10)

In [ ]:
metric=model_inter.evaluate(array_inter_evaluate_x, array_inter_evaluate_y)

2/2 [==============================] - 0s 7ms/step - loss: 0.0660


In [ ]:
model_inter.predict([[32,-80,22]])[0][0]
#32,-80,22,2.140223264694214
#-10,-34,70,0.6837155818939209

1/1 [==============================] - 0s 32ms/step


2.4741075

In [ ]:
model_inter.summary()

# Extrapolieren


In [ ]:
print(len(data.index))

543


In [ ]:
extrapolate_train = data.iloc[:362] 
print('Anzahl der Daten im Trainingsset: '+ str(len(interpolate_train.index)))
print(interpolate_train)


extrapolate_evaluate = data.iloc[:-181] 
print('anzahl der Daten im Evaluateset: '+ str(len(interpolate_evaluate.index)))
print(interpolate_evaluate)

Zum extrapolieren werden in das Train-Datenset die Speed Werte 10 sowie 30 genommen. Das Evaluate-Datenset besteht aus den Daten, die mit Speed 50 aufgenommen wurden. 

In [ ]:
array_extrapolate_train_x = extrapolate_train[['j1_Degree', 'j2_Degree', 'Speed']].to_numpy()
array_extrapolate_train_y = extrapolate_train[['Time']].to_numpy()

array_extrapolate_evaluate_x = extrapolate_evaluate[['j1_Degree', 'j2_Degree', 'Speed']].to_numpy()
array_extrapolate_evaluate_y = extrapolate_evaluate[['Time']].to_numpy()

In [ ]:
model_extra=keras.models.Sequential()

In [ ]:
i = 0
while i < 3:
  model_extra.add(keras.layers.Dense(4,activation='relu'))
  i = i + 1 

In [ ]:
model_extra.add(keras.layers.Dense(1,activation='linear'))

In [ ]:
model_extra.compile(loss='mse',optimizer=tf.keras.optimizers.Adam())

In [ ]:
model_extra.fit(array_inter_train_x,array_inter_train_y,epochs=10,batch_size=10)

In [ ]:
metric=model_extra.evaluate(array_extrapolate_evaluate_x, array_extrapolate_evaluate_y)

12/12 [==============================] - 0s 3ms/step - loss: 0.0940


In [ ]:
model_extra.predict([[60,-20,50]])[0][0]
#60,-20,50,1.0199153423309326

1/1 [==============================] - 0s 69ms/step


0.86125016